In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

# N_DOCUMENTS = 517402
N_DOCUMENTS = 100

In [2]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [3]:
# Load Dataset
X = []
for i in range(1, N_DOCUMENTS):
    f = open("emails/processed/" + str(i + 1) + ".txt", "r")
    clean_string = ""
    for line in f:
        if line[:4] == "From":
            line = line.split(" ")
            clean_string += line[1].split("@")[0] + "\n"
        elif line[:2] == "To":
            line = line.split(" ")
            clean_string += line[1].split("@")[0] + "\n"
        elif line[:7] == "Subject":
            line = line.split(" ")
            clean_string += line[1]
        elif line[:10] == "X-FileName":
            break
    for line in f:
        clean_string += line.strip("\n") + "\n"
    f.close()
    X.append(clean_string)

In [4]:
lemmatizer = WordNetLemmatizer()
for i in range(len(X)):
    sent_text = nltk.sent_tokenize(X[i])
    tokenized_text = []
    stop_words = set(stopwords.words('english')) 
    # Remove the numbers
    for sentence in sent_text:
        sentence = re.sub("[^a-zA-Z]", " ", sentence)
        tokenized_text += list(nltk.word_tokenize(sentence))
        
    # Remove stopwords
    X[i] = [x.lower() for x in tokenized_text if x.lower() not in stop_words and len(x) > 2]
    
    # Pos Tag    
    X[i] = nltk.pos_tag(X[i])
    
    # Lemmatize
    X[i] = list(map(lambda x: lemmatizer.lemmatize(x[0], pos = get_wordnet_pos(x[1])), X[i]))